# Install Required Libraries

In [ ]:
!pip install scikit-learn==1.0.2

In [ ]:
!pip install rdkit-pypi

In [ ]:
!pip install pymatgen

In [ ]:
!pip install nglview

In [ ]:
!pip install ase

In [ ]:
!pip install pandas==1.4.2

# Import Required Libraries/Modules

In [ ]:
# Import Modules

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from pickle import dump, load
import warnings
from IPython.display import display
import urllib.request
import pickle

import rdkit 
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem import rdMolDescriptors as d1
from rdkit.Chem import Descriptors as d2
from rdkit.Chem import AllChem

import sklearn
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 


import pymatgen
from pymatgen.core.structure import Structure
import nglview as nv
import ase

from google.colab import output
output.enable_custom_widget_manager()

# Lecture 5: Closed-loop ML Programs

### ML Workflow:
<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_One/ml_workflow.png?raw=True"/></center>

### The Ideal Closed Loop

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_One/ml_closed_loop.png?raw=True"/></center>

### Genetic Algorithms 

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_One/ga_nature.png?raw=True"/></center>

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_One/ga_molecule.png?raw=True"/></center>

### Need new materials/molecules to evaluate. This can be done in many ways. For example: 

Small Molecules (SMILES)
1. Mutate smiles

2. Fragment (mutate) and combine Smiles 

Inorganic Crystals (CIFs)
1. Substitute anions/cations (ex: I- --> OH-)

2. Dope crystals [isovalent, hetrovalent, interstitial] 

3. Create vacancies  

3. Change cell parameters (lattice params/space group/ strain)

# Small Molecules

## ChatGPT (LLMs)

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_five/chatgpt1.png?raw=True"/></center>

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_five/chatgpt2.png?raw=True"/></center>

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_five/chatgpt3.png?raw=True"/></center>

<center><img src="https://github.com/singhk28/a3md_bc4/blob/main/lecture_five/chatgpt4.png?raw=True"/></center>

## Load path/url for Required Files

In [ ]:
scaler =  'https://github.com/singhk28/a3md_bc4/blob/main/lecture_five/scaler_org.pkl?raw=True'
data_transformed = 'https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_3/data_transformed/organic_train_transformed.csv'

## Create Functions 

In [ ]:
warnings.filterwarnings("ignore")

In [ ]:
# Create a Function to Draw SMILE strings as 2D structures 
def draw_smiles(smiles_list, mols_each_row):
    molecules = [Chem.MolFromSmiles(smi) for smi in smiles_list]
    img = Draw.MolsToGridImage(molecules, molsPerRow=mols_each_row, subImgSize=(400, 400))
    display(img)

In [ ]:
# Create a Function to Create Mutated Version of a List of SMILE Strings 
def mutate_smiles_list(smiles_list, output_length):
    
    # Create a list to hold the mutated SMILE STRINGS 
    mutated_smiles_list = []
    while len(list(set(mutated_smiles_list))) < output_length:
        # Choose a random SMILES string from the input list
        idx = random.randint(0, len(smiles_list)-1)
        original_smiles = smiles_list[idx]
        # Convert the SMILES string to an RDKit molecule object
        mol = Chem.MolFromSmiles(original_smiles)
        # Choose a random atom in the molecule to mutate
        atom_idx = random.randint(0, mol.GetNumAtoms()-1)
        atom = mol.GetAtomWithIdx(atom_idx)
        # Choose a random mutation for the selected atom
        possible_mutations = ['C', 'N', 'O', 'F', 'Cl', 'Br', 'I', 'S']
        possible_mutations.remove(atom.GetSymbol())
        mutation = random.choice(possible_mutations)
        # Perform the mutation
        atom.SetAtomicNum(Chem.GetPeriodicTable().GetAtomicNumber(mutation))
        # Generate a new SMILES string from the mutated molecule
        new_smiles = Chem.MolToSmiles(mol, isomericSmiles=True)
        # Check if the new SMILES string is valid and add it to the output list if it is
        if Chem.MolFromSmiles(new_smiles):
            mutated_smiles_list.append(new_smiles)
            
    return list(set(mutated_smiles_list))

In [ ]:
## Create functions for the Transformation/Featurization Pipeline: 

# 1. Create a function to convert Smiles to RDKIT and then check for errors:
def smiles_to_mol_qa(df: pd.core.frame.DataFrame = None,
                     smiles_col: str = 'smiles',
                     mol_obj_col: str = 'mol_objs') -> pd.core.frame.DataFrame:
    
    
    if df is not None:
        df2 = df.copy()
        
        #Convert smiles to Mol objects:
        df2[mol_obj_col] = df2[smiles_col].apply(lambda smile_string: rdkit.Chem.MolFromSmiles(smile_string))
        df2.dropna(inplace=True)
        df2.reset_index(drop=True)
                
        return df2
    
    else:
        print('Please provide DataFrame variable name') 
        

# 2. Create a function for featurization: 
def small_mol_featurizer(df: pd.core.frame.DataFrame = None,
                         mol_obj_col: str = 'mol_objs',
                         featurizer_dict: dict = {'mol_wt':d1.CalcExactMolWt, 
                                                  'Num_Rings':d1.CalcNumRings, 
                                                  'Aromatic_rings':d1.CalcNumAromaticRings,
                                                  'HBA': d1.CalcNumHBA,
                                                  'HBD': d1.CalcNumHBD,
                                                  'C_SP3_Hybrid': d1.CalcFractionCSP3,
                                                  'Num_heavy_atoms': d1.CalcNumHeavyAtoms,
                                                  'Valence_electrons': d2.NumValenceElectrons}) -> pd.core.frame.DataFrame:
      
    if df is not None:
        df_with_features = df.copy()
        
        for feature in featurizer_dict:
            df_with_features[feature] = df_with_features[mol_obj_col].apply(lambda mol_obj: featurizer_dict[feature](mol_obj))

        return df_with_features, featurizer_dict
    
    
    else:
        print('Please provide DataFranme variable name')

# 3. Create a function for structural fingerprint/featurization:
def morgan_fp(df: pd.core.frame.DataFrame = None,
              mol_obj_col: str = 'mol_objs',
              num_cir: int = 2,
              fp_size: int = 512) -> pd.core.frame.DataFrame:
    
    fp_df_list = []
    
    if df is not None:
        df2 = df.copy()
              
        for mol_obj in df2[mol_obj_col]:
            morg_fp = AllChem.GetMorganFingerprintAsBitVect(mol_obj, num_cir, fp_size)
            morg_fp_string = morg_fp.ToBitString()
            morg_fp_arr = np.fromiter(morg_fp_string, dtype='int')
            fp_df = pd.DataFrame(morg_fp_arr).T
            fp_df_list.append(fp_df)
    
    fp_df = pd.concat(fp_df_list).reset_index(drop=True)
    
    return pd.concat([df2, fp_df], axis=1)  

# 4. Create a function to normalize data
def normalize_data(df: pd.core.frame.DataFrame = None,
                   mode: str = 'create',
                   cols_to_norm: list = None,
                   scaler_filename: str = 'scaler_org.pkl',
                   scaler_url: str = None) -> pd.core.frame.DataFrame:
    
    if df is not None:
           
        if mode == 'create':
            if cols_to_norm is not None:
                # Initialize scaler:
                scaler = MinMaxScaler()
            
                # Fit the Scaler to Data:
                fitted_scaler = scaler.fit(df[cols_to_norm])
                
                # Saving our fitted Scaler for later use:
                dump(fitted_scaler, open(scaler_filename, 'wb'))
                
                # Transform the data using the fitted scaler:
                df[cols_to_norm] = fitted_scaler.transform(df[cols_to_norm])
                
                return df

        if mode == 'use':
            if cols_to_norm is not None:
                loaded_scaler = pickle.load(urllib.request.urlopen(scaler_url, timeout=25))
                df[cols_to_norm] = loaded_scaler.transform(df[cols_to_norm])
                
                return df

# 5. Create a function to normalize data
def clean_up(df: pd.core.frame.DataFrame = None,
             cols_to_drop: list = ['smiles', 'mol_objs']) -> pd.core.frame.DataFrame:
    
    smiles_array = np.array(df['smiles'])
    
    df = df.drop(cols_to_drop, axis=1)
    return df, smiles_array

# 6. Create a pipeline combining all transformation functions:
def transformation_pipeline(df: pd.core.frame.DataFrame = None,
                            normalizer_mode: str = None,
                            scaler_url: str = None)-> pd.core.frame.DataFrame:
    
    
    if df is not None:
    
        # Create a Copy of Df
        df2 = df.copy()    

        # Convert Smiles to Mol Objects and Remove those with Errors
        df2 = smiles_to_mol_qa(df2)

        # Featurize the structures:
        df2, featurizer_dict = small_mol_featurizer(df2)
        df2 = morgan_fp(df2)

        # Normalize Data: 
        if normalizer_mode is not None:
            df2 = normalize_data(df2, normalizer_mode, list(featurizer_dict.keys()), scaler_url=scaler_url)
          
        df2, smiles_array = clean_up(df2)
        
        return df2, smiles_array
    
    else:
        print('Please provide Df to transform')

In [ ]:
def plot_actual_vs_predicted(actual, predicted):
  """
  DOC STRING
  """
  data = np.arange(min(actual), max(actual)+1)

  plt.scatter(actual, predicted)
  plt.plot(data,data, color='red', linestyle='dashed')
  plt.xlabel('Actual Value')
  plt.ylabel('Predicted Value')
  plt.title('Actual vs. Predicted Values')
  plt.show() 

In [ ]:
# Create a cyclic search function 
def cyclic_search(list_of_smiles: list = None,
                  desired_value: float = None,
                  threshold_value: float = None,
                  trained_model: sklearn.linear_model._ridge.Ridge = None,
                  scaler_url: str = None,
                  n_select: int = 5,
                  iterations: int = 10) -> pd.core.frame.DataFrame:

    """
    
    Function iteratively generates mutated SMILE strings and predicts bandagap.
    The function will stop iterating when the max number of allowed iterations 
    are reached or when the desired value is obtained within the provided 
    threshold value. 
    
    """
    
    
    pool_of_smiles = list_of_smiles
    selected_structures = pd.DataFrame()

    for iteration in range(0,iterations): 
        # Generate a DF with SMILES based on mutated versions of input SMILES list:
        list_of_new_smiles = mutate_smiles_list(pool_of_smiles, 50)
        df = pd.DataFrame(list_of_new_smiles, columns=['smiles'])

        # Apply transformation pipeline to the SMILES DF:
        df2, smiles_array = transformation_pipeline(df, 'use', scaler_url)

        # Generate predictions using the trained model:
        df['predicted_bandgap'] = trained_model.predict(np.array(df2))

        # Compare results to desired value and threshold:
        df_subset = df[(df['predicted_bandgap'] >= (desired_value-threshold_value))
                       & (df['predicted_bandgap'] <= (desired_value+threshold_value))]

        if len(selected_structures) > 0:
            # Calculate differences in predicted value and desired value and sort by smallest differences 
            df_subset['abs_diff'] = np.absolute(desired_value - df_subset['predicted_bandgap'])
            df_subset = df_subset.sort_values(['abs_diff'])
            
            if len(df_subset) >= n_select:
                # Take the top n_select from subset:
                top_picks = df_subset[0:n_select]
                selected_structures = selected_structures.append(top_picks)
                n_selected_df = selected_structures.sort_values(['abs_diff'])
                n_selected_df = n_selected_df[:n_select]  
                return n_selected_df.reset_index(drop=True)
            
            else:
                top_picks = df_subset
                selected_structures = selected_structures.append(top_picks)
                pool_of_smiles = list(np.random.choice(smiles_array, 25))
       
        if len(df_subset) > 0:
            # Calculate differences in predicted value and desired value and sort by smallest differences 
            df_subset['abs_diff'] = np.absolute(desired_value - df_subset['predicted_bandgap'])
            df_subset = df_subset.sort_values(['abs_diff'])
            
            if len(df_subset) >= n_select:
                # Take the top n_select from subset:
                top_picks = df_subset[0:n_select]
                return top_picks.reset_index(drop=True)
                
            else:
                top_picks = df_subset
                selected_structures = selected_structures.append(top_picks)
                pool_of_smiles = list(np.random.choice(smiles_array, 25))
        
        if len(df_subset) == 0:
            pool_of_smiles = list(np.random.choice(smiles_array, 25))
            
    if len(selected_structures) == 0:
        print('Could not find any structures satisfying the provided criteria in the allowed cycles.')
    
    return selected_structures.reset_index(drop=True)

## Create Model on Trained Data:

### Load Data:

#### Exercise: 

1. Load the transformed data as a pandas dataframe.
2. What is the maximum and minimum value for the bandgap in our dataset?  

In [ ]:
# YOUR CODE HERE 

In [ ]:
df = pd.read_csv(data_transformed)
df

In [ ]:
# YOUR CODE HERE 

In [ ]:
print('min value for bandgap: ', df.bandgap.min())
print('max value for bandgap: ', df.bandgap.max())

### Create a Model

#### Exercise:

1. Create a Linear (Ridge) Model. Remember to tune the alpha value.  
2. Make a prediction on two molecules from the [PhotochemCAD Database](https://www.photochemcad.com/). 

In [ ]:
# PhotochemCAD Data

photo_cad_dict = {'dye_names':['Coumarin', 'Porphyrin_h2p', 'DCM', 'Sudan4', 'PDI'],
                  'smiles':['C1=CC=C2C(=C1)C=CC(=O)O2',
                            'C1(/C=C(N/2)/C=CC2=C\C(C=C/3)=NC3=C/4)=N/C(C=C1)=C\C5=CC=C4N5',
                            'CC(OC(/C=C/C1=CC=C(N(C)C)C=C1)=C/2)=CC2=C(C#N)/[N+]#[C-]',
                            'OC1=CC=C2C(C=CC=C2)=C1/N=N/C3=C(C)C=C(/N=N/C4=C(C)C=CC=C4)C=C3',
                            'O=C(C1=C2C(C3=CC=C24)=C(C5=C6C3=CC=C7C6=C(C(N(C8=C(C(C)(C)C)C=CC(C(C)(C)C)=C8)C7=O)=O)C=C5)C=C1)N(C9=C(C(C)(C)C)C=CC(C(C)(C)C)=C9)C4=O'],
                   'dye_abs_values':[4.50, 3.13, 2.65, 2.39, 2.35]}

In [ ]:
# Create X and Y Data Arrays:
X = df_train[df_train.columns[1:]]
y = df_train[df_train.columns[0]]

# Split data into train and test set:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

# Train A Ridge Regression Model w/ GridSearch:

# Initialize model:
ridge_mod = Ridge()

# Create a dictionary of all values we want to test for n_neighbors:
parameter_grid = {'alpha':np.arange(0.5,10.5,0.5)}

# Initialize GridSearch:
ridge_mod_gscv = GridSearchCV(ridge_mod, parameter_grid, cv=5)

# Fit the Various Parameter Models and find the best one:
ridge_mod_gscv.fit(X_train, y_train)
print('Best parameters :', ridge_mod_gscv.best_params_)
best_ridge_mod = ridge_mod_gscv.best_estimator_

# Generate predictions w/ fitted model:
predictions = best_ridge_mod.predict(X_test)

# Measure performance:
from sklearn.metrics import mean_absolute_error
print('MAE :', mean_absolute_error(y_test, predictions))

# Plot the predictions:
plot_actual_vs_predicted(y_test, predictions)

In [ ]:
# Photochemcad dataset 
# Create a DataFrame
df_cad = pd.DataFrame(photo_cad_dict)
df_cad_ml = df_cad[['smiles']]

# Apply transformation pipeline to the SMILES DF:
df_cad_ml_feat, smiles_array = transformation_pipeline(df_cad_ml, 'use', scaler)

# Generate predictions using the trained model:
df['predicted_bandgap'] = best_ridge_mod.predict(np.array(df_cad_ml_feat))
df

In [ ]:
# Look @ Structures in our initial_smiles_list:
draw_smiles(df['smiles'], 1)

## Finding Molecules that Emit According to the REC 2020 Wavelength: 


In [ ]:
red_mols = cyclic_search(df['smiles'], 1.97, 0.2, best_ridge_mod, scaler, n_select=5)  
red_mols

In [ ]:
draw_smiles(red_mols['smiles'],3)

In [ ]:
list(red_mols['smiles'][:])

Evaluate the synthesizability of the generated molecules. We will use the ASKCOS [SCScore Evaluator](https://askcos.mit.edu/scscore/). 

### Exercise:

1. Generate a list of 5 smiles with absorption in the blue region (2.65 eV).
2. Which structure has the best synthesizability. 

In [ ]:
# YOUR CODE HERE

# Inorganic Crystals

In [ ]:
# Load data and pick a structure
crystal_data = pd.read_csv("https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_3/data_transformed/inorganic_featurized_cif.csv")

crystal = Structure.from_str(str(crystal_data.iloc[2]["cif"]), fmt="cif")
print(crystal)

## Visualize Structure


In [ ]:
view = nv.show_pymatgen(crystal)
view.add_unitcell()
view

NGLWidget()

## Crystal element substitution

In [ ]:
from pymatgen.transformations.standard_transformations import SubstitutionTransformation

# Create and apply substitution transformation
subst_transf = SubstitutionTransformation({"Li": "Cu"})
crystal_subt = subst_transf.apply_transformation(crystal)

print("Crystal after substitution: ")
print(crystal_subt)
view = nv.show_pymatgen(crystal_subt)
view.add_unitcell()
view

##  Insert new atoms

In [ ]:
from pymatgen.transformations.site_transformations import InsertSitesTransformation

insert_transf = InsertSitesTransformation(["Ce", "Fe"], coords=[[0.12,0.3,0.1],[0.25,0.2,0.5]])
crystal_insert = insert_transf.apply_transformation(crystal)
print(crystal_insert)
view = nv.show_pymatgen(crystal_insert)
view.add_unitcell()
view


## Crystal doping

In [ ]:
from pymatgen.transformations.site_transformations import ReplaceSiteSpeciesTransformation
import random


def get_doped_structure(structure: pymatgen.core.structure, target_element: str, dopant: str, fraction: float):
    """
    This function performs doping of provided structure

    Arguments:
       structure (pymatgen.core.structure): inorganic crystal for doping
       target_element (str): element of the structure that will be substituted 
       dopant (str): element that will be used to substitute target element
       fraction (float): fraction of atoms of the structure that will be substituted
       
    Returns: 
       Doped structure pymatgen.core.structure
       (None if fraction is not within 0-1 range)
    """

    # Check if fraction within 0-1 range
    if fraction < 0.0 or fraction > 1.0:
      print("Please provide fraction within [0, 1] range")
      return
    
    n_target_elements = 0
    target_element_idx_list = []
    for idx, site in enumerate(structure):
        if str(site.specie) == target_element:
            n_target_elements+=1
            target_element_idx_list.append(idx)
    
    # Retunrn unchanged if no target elements are present
    if n_target_elements == 0:
      print("Provided structure does not contain target element for doping. The unchanged structure is returned")
      return structure

    # Calculate number of atoms to substitute based on fraction value and number of target atoms in the structure 
    n_substitutions = int(n_target_elements * fraction)
    sites_to_replace = random.sample(target_element_idx_list, n_substitutions)

    # Create a list of atoms that will be placed instead of target atoms  
    dopant_list = [dopant]*len(sites_to_replace)
    replace_dictionary = dict(zip(sites_to_replace, dopant_list))
    # print(replace_dictionary)

    # Reolace seleccted atoms and return a doped structure
    replace_species_transf = ReplaceSiteSpeciesTransformation(replace_dictionary)
    doped_structure = replace_species_transf.apply_transformation(structure)
    
    return doped_structure
    


In [ ]:
random.seed(123)
doped_crystal = get_doped_structure(crystal, target_element="O", dopant="Mg", fraction=0.5)
print(doped_crystal)

view = nv.show_pymatgen(doped_crystal)
view.add_unitcell()
view

## Diffusion models

<center><img src="https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_5/15.jpg"/></center>

<center><img src="https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_5/16.jpg"/></center>

Code: https://github.com/ehoogeboom/e3_diffusion_for_molecules
<center><img src="https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_5/17.jpg"/></center>

Code: https://github.com/MinkaiXu/GeoDiff
<center><img src="https://raw.githubusercontent.com/ineporozhnii/a3md_bootcamp/main/day_5/18.jpg"/></center>